In [33]:
import re
import requests
from bs4 import BeautifulSoup

In [34]:
CLASS_URL = ("https://udapps.nss.udel.edu/CoursesSearch/search-results" 
"?term=2183"
"&search_type=A"
"&course_sec=CHEM"
"&session=All"
"&course_title="
"&instr_name="
"&text_info=All"
"&instrtn_mode=All"
"&time_start_hh="
"&time_start_ampm="
"&credit=Any"
"&keyword="
"&subj_area_code="
"&college="
"&startat="
)
current_url = CLASS_URL
HEADERS = {'User-Agent': 'Mozilla/5.0'}
print CLASS_URL
def get_class_page(url):
    response = requests.get(url, headers=HEADERS)
#     print 'response status: ', response.status_code
    return BeautifulSoup(response.content, 'lxml'), response.status_code

https://udapps.nss.udel.edu/CoursesSearch/search-results?term=2183&search_type=A&course_sec=CHEM&session=All&course_title=&instr_name=&text_info=All&instrtn_mode=All&time_start_hh=&time_start_ampm=&credit=Any&keyword=&subj_area_code=&college=&startat=


In [36]:
current_page, current_status = get_class_page(current_url)
print current_status
print current_page.find(id='searchNxtBtn')

200
<button class="btn btn-default btn-block" id="searchNxtBtn" onclick="location.href='search-results?&amp;term=2183&amp;search_type=A&amp;course_sec=CHEM&amp;text_info=All&amp;credit=Any&amp;session=All&amp;instrtn_mode=All&amp;startat=2183CHEM104064';" type="button">Next <span class="glyphicon glyphicon-arrow-right"></span></button>


In [37]:
def find_table(page):
    return page.find_all('table')[0]

In [38]:
table = find_table(current_page)

In [39]:
def find_table_headers(table):
    return [th.text.encode('ascii') for th in table.select('th')]



In [40]:
table_headers = find_table_headers(table)
table_headers

['Course',
 'Title',
 'Open seats',
 'Credits',
 'Day',
 'Time',
 'Location',
 'Instructor']

In [41]:
def get_table_data(table):
    """
    :param: table: bs4 table element
    :returns: a list of lists of td elements, analogous to a list of spreadsheet rows"""
    table_rows = table.select('tr')
    table_data = [[td.text.strip().encode('ascii') for td in row.select('td')]
             for row in table_rows[1:]]
    return table_data



In [42]:
print get_table_data(table)

[['CHEM100010\nLEC', 'Chemistry and Human Environment', '1 OF 60', '3 Hrs', 'TR', '12:30PM - 1:45PM', '', 'Ridge,Douglas P'], ['CHEM100510\nLEC', 'Chemistry and Human Environment', '31 OF 35', '3 Hrs', 'TR', '5:30PM - 6:45PM', '', 'Pascucci,Pete'], ['CHEM101010\nLEC', 'General Chemistry', '5 OF 144', '4 Hrs', 'TR', '12:30PM - 1:45PM', '', 'Gnanamgari,Jennifer Wood'], ['CHEM101020L\nLAB', 'General Chemistry', '0 OF 24\n                  \n                  \n                  \n                   CURRENTLY FULL', '4 Hrs', 'M', '7:00PM - 10:00PM', '', 'Gnanamgari,Jennifer Wood'], ['CHEM101021L\nLAB', 'General Chemistry', '0 OF 24\n                  \n                  \n                  \n                   CURRENTLY FULL', '4 Hrs', 'M', '12:20PM - 3:20PM', '', 'Gnanamgari,Jennifer Wood'], ['CHEM101022L\nLAB', 'General Chemistry', '0 OF 24\n                  \n                  \n                  \n                   CURRENTLY FULL', '4 Hrs', 'R', '3:30PM - 6:30PM', '', 'Gnanamgari,Jen

In [47]:
def get_next_url(page, current_url):
    next_button = page.find(id='searchNxtBtn')
    if next_button is None:
        print 'end reached'
        return None
    print 'current url: ', current_url
    next_url_hint = next_button['onclick']
    startat = re.compile('&startat=[^&\']*')
    print 'searching: ', next_url_hint
    new_startat = startat.search(next_url_hint).group()
    print 'new startat: ', new_startat

    new_url = startat.sub(new_startat, current_url)
    print 'new url: ', new_url
    return new_url
    

In [48]:
print current_url
# print current_page.find(id='searchNxtBtn')
print 'current page button onclick:', current_page.find(id='searchNxtBtn')['onclick']
page_two_url = get_next_url(current_page, current_url)
print 'page two url: ', page_two_url
page_two, status = get_class_page(page_two_url)
# page_two_button = page_two.find(id='searchNxtBtn')['onclick']
print "button 2: ", page_two.find(id='searchNxtBtn')['onclick']
page_three_url = get_next_url(page_two, page_two_url)
print 'page three url: ', page_three_url

https://udapps.nss.udel.edu/CoursesSearch/search-results?term=2183&search_type=A&course_sec=CHEM&session=All&course_title=&instr_name=&text_info=All&instrtn_mode=All&time_start_hh=&time_start_ampm=&credit=Any&keyword=&subj_area_code=&college=&startat=
current page button onclick: location.href='search-results?&term=2183&search_type=A&course_sec=CHEM&text_info=All&credit=Any&session=All&instrtn_mode=All&startat=2183CHEM104064';
current url:  https://udapps.nss.udel.edu/CoursesSearch/search-results?term=2183&search_type=A&course_sec=CHEM&session=All&course_title=&instr_name=&text_info=All&instrtn_mode=All&time_start_hh=&time_start_ampm=&credit=Any&keyword=&subj_area_code=&college=&startat=
searching:  location.href='search-results?&term=2183&search_type=A&course_sec=CHEM&text_info=All&credit=Any&session=All&instrtn_mode=All&startat=2183CHEM104064';
new startat:  &startat=2183CHEM104064
new url:  https://udapps.nss.udel.edu/CoursesSearch/search-results?term=2183&search_type=A&course_sec=C

In [49]:
def process_pages(current_page, current_table, current_url, data):
    data.append(get_table_data(current_table))
    new_url = get_next_url(current_page, current_url)
    if new_url == None:
        return
    else:
        print 'old url: ', current_url[-25:]
        print 'new_url: ', new_url[-25:]
        new_page, status = get_class_page(new_url)
        new_table = find_table(new_page)
        process_pages(new_page, new_table, new_url, data)
                

In [52]:
def scrape_pages(current_url):
    current_page, status = get_class_page(current_url)
    all_data = []
#     at_end = False
    current_table = find_table(current_page)
    headers = find_table_headers(current_table)
    print 'headers: ', headers
    process_pages(current_page, current_table, current_url, all_data)
    
    return headers, all_data
        
        
    

In [53]:
headers, data = scrape_pages(current_url)
print headers
print data[:3]
print data[-3:]

headers:  ['Course', 'Title', 'Open seats', 'Credits', 'Day', 'Time', 'Location', 'Instructor']
current url:  https://udapps.nss.udel.edu/CoursesSearch/search-results?term=2183&search_type=A&course_sec=CHEM&session=All&course_title=&instr_name=&text_info=All&instrtn_mode=All&time_start_hh=&time_start_ampm=&credit=Any&keyword=&subj_area_code=&college=&startat=
searching:  location.href='search-results?&term=2183&search_type=A&course_sec=CHEM&text_info=All&credit=Any&session=All&instrtn_mode=All&startat=2183CHEM104064';
new startat:  &startat=2183CHEM104064
new url:  https://udapps.nss.udel.edu/CoursesSearch/search-results?term=2183&search_type=A&course_sec=CHEM&session=All&course_title=&instr_name=&text_info=All&instrtn_mode=All&time_start_hh=&time_start_ampm=&credit=Any&keyword=&subj_area_code=&college=&startat=2183CHEM104064
old url:  a_code=&college=&startat=
new_url:  e=&startat=2183CHEM104064
current url:  https://udapps.nss.udel.edu/CoursesSearch/search-results?term=2183&search_ty

In [63]:
def convert_page_data(data):
    result = []
    for page in data:
        for row in page:
            result.append(row)
    return result

In [61]:
data

[[['CHEM100010\nLEC',
   'Chemistry and Human Environment',
   '1 OF 60',
   '3 Hrs',
   'TR',
   '12:30PM - 1:45PM',
   '',
   'Ridge,Douglas P'],
  ['CHEM100510\nLEC',
   'Chemistry and Human Environment',
   '31 OF 35',
   '3 Hrs',
   'TR',
   '5:30PM - 6:45PM',
   '',
   'Pascucci,Pete'],
  ['CHEM101010\nLEC',
   'General Chemistry',
   '5 OF 144',
   '4 Hrs',
   'TR',
   '12:30PM - 1:45PM',
   '',
   'Gnanamgari,Jennifer Wood'],
  ['CHEM101020L\nLAB',
   'General Chemistry',
   '0 OF 24\n                  \n                  \n                  \n                   CURRENTLY FULL',
   '4 Hrs',
   'M',
   '7:00PM - 10:00PM',
   '',
   'Gnanamgari,Jennifer Wood'],
  ['CHEM101021L\nLAB',
   'General Chemistry',
   '0 OF 24\n                  \n                  \n                  \n                   CURRENTLY FULL',
   '4 Hrs',
   'M',
   '12:20PM - 3:20PM',
   '',
   'Gnanamgari,Jennifer Wood'],
  ['CHEM101022L\nLAB',
   'General Chemistry',
   '0 OF 24\n                  \n      

In [64]:
good_data = convert_page_data(data)
good_data[0]

['CHEM100010\nLEC',
 'Chemistry and Human Environment',
 '1 OF 60',
 '3 Hrs',
 'TR',
 '12:30PM - 1:45PM',
 '',
 'Ridge,Douglas P']

In [68]:
len(headers)

8

In [70]:
import pandas as pd
df = pd.DataFrame.from_records(good_data, columns=headers)
df.head(10)

,Course,Title,Open seats,Credits,Day,Time,Location,Instructor
0,CHEM100010\nLEC,Chemistry and Human Environment,1 OF 60,3 Hrs,TR,12:30PM - 1:45PM,,"Ridge,Douglas P"
1,CHEM100510\nLEC,Chemistry and Human Environment,31 OF 35,3 Hrs,TR,5:30PM - 6:45PM,,"Pascucci,Pete"
2,CHEM101010\nLEC,General Chemistry,5 OF 144,4 Hrs,TR,12:30PM - 1:45PM,,"Gnanamgari,Jennifer Wood"
3,CHEM101020L\nLAB,General Chemistry,0 OF 24\n \n ...,4 Hrs,M,7:00PM - 10:00PM,,"Gnanamgari,Jennifer Wood"
4,CHEM101021L\nLAB,General Chemistry,0 OF 24\n \n ...,4 Hrs,M,12:20PM - 3:20PM,,"Gnanamgari,Jennifer Wood"
5,CHEM101022L\nLAB,General Chemistry,0 OF 24\n \n ...,4 Hrs,R,3:30PM - 6:30PM,,"Gnanamgari,Jennifer Wood"
6,CHEM101023L\nLAB,General Chemistry,0 OF 24\n \n ...,4 Hrs,W,12:20PM - 3:20PM,,"Gnanamgari,Jennifer Wood"
7,CHEM101024L\nLAB,General Chemistry,3 OF 24,4 Hrs,F,12:20PM - 3:20PM,,"Gnanamgari,Jennifer Wood"
8,CHEM101025L\nLAB,General Chemistry,2 OF 24,4 Hrs,F,3:35PM - 6:35PM,,"Gnanamgari,Jennifer Wood"
9,CHEM101510\nLEC\nWL,General Chemistry,13 OF 16,4 Hrs,TR,7:00PM - 8:15PM,,"Pascucci,Pete"


In [71]:
df.to_csv('test_spring18.csv')

In [ ]:
first_class = table_rows[1]
tds = first_class.select('td')
row_data = [td.text.strip().encode('ascii') for td in tds]
row_data
# for td in first_class.select('td'):
#     print td.text.strip()
    

In [ ]:
table_data = [[td.text.strip().encode('ascii') for td in row.select('td')]
             for row in table_rows[1:]]
table_data

In [ ]:
next_button = first_page.find(id='searchNxtBtn')
next_url_hint = next_button['onclick']
next_url_hint

In [ ]:
import re
startat = re.compile('&startat=[^&]*')
new_startat = startat.search(next_url_hint).group()
new_startat

In [ ]:
# old_startat = find_startat.search(current_url)
new_url = startat.sub(new_startat, current_url)
new_url

In [ ]:
BASE_URL = 'http://en.wikipedia.org'
HEADERS = {'User-Agent': 'Mozilla/5.0'}

def get_Nobel_soup():
    response = requests.get(
    BASE_URL + '/wiki/List_of_Nobel_Laureates',
    headers=HEADERS)
    return BeautifulSoup(response.content, 'lxml')

In [ ]:
soup = get_Nobel_soup()

In [ ]:
table = soup.select_one('table.sortable.wikitable')

In [ ]:
table.select('th')

In [ ]:
response.text

In [ ]:
def get_column_titles(table):
    cols = []
    for th in table.select_one('tr').select('th')[1:]:
        link = th.select_one('a')
        if link:
            cols.append({'name': link.text, 'href': link.attrs['href']})
        else:
            cols.append({'name': th.text, 'href': None})
    return cols

In [ ]:
get_column_titles(table)

In [ ]:
def get_Nobel_winners(table):
    cols = get_column_titles(table)
    winners = []
    for row in table.select('tr')[1:-1]:
        year = int(row.select_one('td').text[:4])  # in case year has footnote
        for i, td in enumerate(row.select('td')[1:]):
            for winner in td.select('a'):
                href = winner.attrs['href']
                if not href.startswith('#endnote'):
                    winners.append({
                        'year': year,
                        'category': cols[i]['name'],
                        'name': winner.text,
                        'link': winner.attrs['href']
                    })
    return winners

In [ ]:
get_Nobel_winners(table)